In [1]:
import pandas as pd
import numpy as np
import wfdb
import ast

import wfdb.processing

In [2]:
path = "data/"
Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

# Load scp_statements.csv for diagnostic aggregation
agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

def compute_bpm(df, path):
    HB = []
    for idx, f in enumerate(df.filename_lr):
        sig, fields = wfdb.rdsamp(path+f)
        qrs_inds = [wfdb.processing.XQRS(sig=sig[:,ch_idx],fs=fields['fs']) for ch_idx in range(12)]
        for qr in qrs_inds:
            qr.detect(verbose=False)
        
        intervals = [wfdb.processing.calc_rr(qrs_inds[ch_idx].qrs_inds) for ch_idx in range(12)]
        bpm = [wfdb.processing.calc_mean_hr(intervals[ch_idx], fs=fields["fs"]) for ch_idx in range(12)]
        
        if idx % 100 == 0: print(f"Processed {idx} samples")
        HB += [bpm]
    return HB

BPM = compute_bpm(Y, path)

Processed 0 samples
Processed 100 samples
Processed 200 samples
Processed 300 samples
Processed 400 samples
Processed 500 samples
Processed 600 samples
Processed 700 samples
Processed 800 samples
Processed 900 samples
Processed 1000 samples
Processed 1100 samples
Processed 1200 samples
Processed 1300 samples
Processed 1400 samples
Processed 1500 samples
Processed 1600 samples
Processed 1700 samples
Processed 1800 samples
Processed 1900 samples
Processed 2000 samples
Processed 2100 samples
Processed 2200 samples
Processed 2300 samples
Processed 2400 samples
Processed 2500 samples
Processed 2600 samples
Processed 2700 samples
Processed 2800 samples
Processed 2900 samples
Processed 3000 samples
Processed 3100 samples
Processed 3200 samples
Processed 3300 samples
Processed 3400 samples
Processed 3500 samples
Processed 3600 samples
Processed 3700 samples
Processed 3800 samples
Processed 3900 samples
Processed 4000 samples
Processed 4100 samples
Processed 4200 samples
Processed 4300 samples


In [25]:
print(np.shape(BPM))
BPM = np.array(BPM)

(21801, 12)


In [26]:
for i in range(1, 11):
    name = f"data/nparrays/bpm/{str(i).zfill(2)}"
    X_ = BPM[np.where(Y.strat_fold == i)[0]]
    np.save(name, X_)